# Extracting Spectra

This notebook demonstrates how to grab 1D aperture summed HETDEX spectra for an input of ID, RA and DEC using the `Extract` Class API from `hetdex_api`. This can be done interactively using `get_spectra` from hte `hetdex_tools.get_spec` module. It can also be done in the command line using the quick entry call `hetdex_get_spec` providing you have hetdex_api pip installed.

Examples of what you might like to do with the spectra afterwards is shown later. The output is stored in an astropy table of spectra. For every HETDEX observation where spectra is found, a spectra is given. It is up to the user to combine the spectra afterwards.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import astropy.units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.table import Table, join, vstack

from hetdex_tools.get_spec import get_spectra

In [ ]:
%matplotlib inline

## Getting all spectra at a specified RA/DEC. This will search through all shots in HDR2.1

If a shotid is not specified the program will search for any shot within HDR2 that overlaps within an 11 arcmin radius of the input coordinates. Because of the non-contiguous VIRUS footprint, there is no guarantee the aperture defined by the input ra/dec/rad will contain enough fibers to do a measurement. The aperture radius is 3" by default or can be specified with the --rad argument. 

Open a catalog of IDs, RAs, DECs:

In [ ]:
input_cat = Table.read('api-notebooks/highz-emitters.fits')

In [ ]:
input_cat[0:5]

`get_spectra()` requires an astropy coordinates object list as an input.

In [ ]:
input_coords = SkyCoord(ra=input_cat['RA'], dec=input_cat['DEC'])

In [ ]:
# we'll just do the first 20 as an example. Many sources are in COSMOS so we have lots of observations with potential coverage
sources = get_spectra(input_coords[0:20], ID=input_cat['ID'][0:20])

##  get_spectra() options

There are a few options to consider when running get_spectra():

In [ ]:
help(get_spectra)

### Reading in the output - astropy FITS files

Join your input and output table so you can match up any properties you like

In [ ]:
output_table = join(input_cat, sources)

In [ ]:
output_table[0:5]

In [ ]:
sel = output_table['FLUX'] > 10

for row in output_table[sel][4:8]:
    plt.figure()
    wave_obj = row['WAVE']
    wave = row['wavelength']
    spec = row['spec']
    spec_err = row['spec_err']
    plt.errorbar(wave, spec, yerr=spec_err)
    plt.xlim(wave_obj-50, wave_obj+50)
    plt.xlabel('wave')
    plt.ylabel('spec')
    plt.title(row['ID'])


Some tools for fitting lines and making a line catalog exist in `hetdex_tools/line_fitting.py` but it is in heavy development still. Examples to come!

Examples of running get_spec as a command line job are shown in api-notebooks/14-Extracting_a_sourcelist.ipynb. For large jobs, you will likely want to submit a batch job on a more powerful cluster (such as stampede2). 